In [10]:
import time
from timeout import timeout
from src.preprocessing import create_audio_dataset, upload_to_huggingface
codebase_dir = '/om2/user/azain/code/voice_anonymization'
transcript_path_pattern = "{base_name}.original.txt"
hf_dataset_name = "azain/LibriTTS"
raw_data_path = f'{codebase_dir}/data/raw/LibriTTS/dev-clean'
processed_data_path = f"{codebase_dir}/tmp/LibriTTS-processed"

In [11]:
@timeout(300)
def time_limited_push_to_hub(dataset, dataset_name):
    dataset.push_to_hub(dataset_name)

In [12]:
def binary_search_upload(dataset, low, high):
    while low <= high:
        mid = (low + high) // 2
        test_dataset = dataset.shuffle(seed=42).select(range(mid))
        
        try:
            time_limited_push_to_hub(test_dataset, f'azain/LibriTTS-{mid}-samples')
            print(f"Upload of size {mid} successful.")
            low = mid + 1  # Attempt a larger size since this size was successful
        except TimeoutError:
            print(f"Upload of size {mid} timed out.")
            high = mid - 1  # Reduce the size and try again
        except Exception as e:
            print(f"Upload failed for size {mid}: {str(e)}")
            high = mid - 1  # Reduce the size due to other exceptions

    return high  # The largest successful upload size under the time constraint

loading files from /om2/user/azain/code/voice_anonymization/data/raw/LibriTTS/dev-clean...
done creating audio dataset


Uploading the dataset shards: 100%|██████████| 1/1 [02:41<00:00, 161.28s/it]


Upload of size 450 successful.


Uploading the dataset shards:   0%|          | 0/1 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
audio_dataset = create_audio_dataset(processed_data_path, transcript_path_pattern)
max_size = binary_search_upload(audio_dataset, 1, 900)
print(f"The largest dataset size that can be successfully uploaded is {max_size} samples.")